# Preparation

In [115]:
import os
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from wordcloud import WordCloud
import glob 
import collections
import MeCab

# Get title and URLS

In [116]:
genres = ["domestic", "world", "business", "entertainment", "sports", "it", "science", "local"]

In [117]:
def make_URL(idx: int, jan: str) -> str:
    return f'https://news.yahoo.co.jp/topics/{jan}?page={idx}'

URLexample = make_URL(1, "domestic")
print(URLexample)

https://news.yahoo.co.jp/topics/domestic?page=1


In [118]:
URL_ = URLexample
rest = requests.get(URL_)

soup = BeautifulSoup(rest.text, "html.parser")

data_list = soup.find_all(href=re.compile("news.yahoo.co.jp/pickup"))

In [119]:
def get_title_url(URL: str, concat: dict) -> [dict, bool]:
    URL_ = URL
    rest = requests.get(URL_)

    soup = BeautifulSoup(rest.text, "html.parser")

    data_list = soup.find_all(href=re.compile("news.yahoo.co.jp/pickup"))
    if len(data_list) == 0:
        continue_ = False
    else:
        for data in data_list:
            if data.div:
                concat[data.find_all('div')[3].string] = [data.attrs["href"], data.find_all('div')[4].string]
            continue_ = True
    return concat, continue_

concat = {}
concat, continue_ = get_title_url(URLexample, concat)
print(concat)
print(continue_)

{'流行語大賞「村神様」に決定': ['https://news.yahoo.co.jp/pickup/6446369', '12/1(木) 14:10'], '全国的に気温低く 北日本は大雪も': ['https://news.yahoo.co.jp/pickup/6446353', '12/1(木) 12:13'], '同性婚巡り 本質に踏み込む判決': ['https://news.yahoo.co.jp/pickup/6446331', '12/1(木) 8:19'], '長射程ミサイル 同時開発を検討': ['https://news.yahoo.co.jp/pickup/6446327', '12/1(木) 8:16'], '敵基地攻撃能力 自公が実質合意': ['https://news.yahoo.co.jp/pickup/6446326', '12/1(木) 7:51'], '冬本番の寒さ 北日本は大雪に警戒': ['https://news.yahoo.co.jp/pickup/6446319', '12/1(木) 6:24'], '愛子さま21歳、コロナ収束に願い': ['https://news.yahoo.co.jp/pickup/6446317', '12/1(木) 0:16'], '江沢民氏死去 岸田氏「哀悼の意」': ['https://news.yahoo.co.jp/pickup/6446301', '11/30(水) 20:31'], '風流踊 ユネスコの無形文化遺産に': ['https://news.yahoo.co.jp/pickup/6446293', '11/30(水) 19:09'], '杉田氏 「洗脳教育」投稿を撤回': ['https://news.yahoo.co.jp/pickup/6446291', '11/30(水) 19:02'], '首相 五輪談合事件に「誠に遺憾」': ['https://news.yahoo.co.jp/pickup/6446282', '11/30(水) 17:44'], '同性婚不許可は違憲状態 東京地裁': ['https://news.yahoo.co.jp/pickup/6446272', '11/30(水) 16:44'], '過少記載疑い 自民・薗浦氏が謝罪': ['

In [120]:
def  make_gen_title_url(jan: str) -> dict:
    continue_ = True
    idx = 1
    concat = {}
    while continue_ != False:
        URL = make_URL(idx, jan)
        concat, continue_ = get_title_url(URL, concat)
        idx += 1
    return concat

concat = make_gen_title_url('domestic')
print(concat)

{'流行語大賞「村神様」に決定': ['https://news.yahoo.co.jp/pickup/6446369', '12/1(木) 14:10'], '全国的に気温低く 北日本は大雪も': ['https://news.yahoo.co.jp/pickup/6446353', '12/1(木) 12:13'], '同性婚巡り 本質に踏み込む判決': ['https://news.yahoo.co.jp/pickup/6446331', '12/1(木) 8:19'], '長射程ミサイル 同時開発を検討': ['https://news.yahoo.co.jp/pickup/6446327', '12/1(木) 8:16'], '敵基地攻撃能力 自公が実質合意': ['https://news.yahoo.co.jp/pickup/6446326', '12/1(木) 7:51'], '冬本番の寒さ 北日本は大雪に警戒': ['https://news.yahoo.co.jp/pickup/6446319', '12/1(木) 6:24'], '愛子さま21歳、コロナ収束に願い': ['https://news.yahoo.co.jp/pickup/6446317', '12/1(木) 0:16'], '江沢民氏死去 岸田氏「哀悼の意」': ['https://news.yahoo.co.jp/pickup/6446301', '11/30(水) 20:31'], '風流踊 ユネスコの無形文化遺産に': ['https://news.yahoo.co.jp/pickup/6446293', '11/30(水) 19:09'], '杉田氏 「洗脳教育」投稿を撤回': ['https://news.yahoo.co.jp/pickup/6446291', '11/30(水) 19:02'], '首相 五輪談合事件に「誠に遺憾」': ['https://news.yahoo.co.jp/pickup/6446282', '11/30(水) 17:44'], '同性婚不許可は違憲状態 東京地裁': ['https://news.yahoo.co.jp/pickup/6446272', '11/30(水) 16:44'], '過少記載疑い 自民・薗浦氏が謝罪': ['

# Loop for getting title and URL

In [121]:
concat_genre = {}
for genre in genres:
    concat = make_jan_title_url(genre)
    concat_genre[genre] = concat

In [21]:
URLexample = list(concat_genre['domestic'].values())[5]

In [22]:
URLexample

['https://news.yahoo.co.jp/pickup/6446319', '12/1(木) 6:24']

# Extract document

In [30]:
def get_document(URL: str) -> [str, int]:
    html_doc = requests.get(URL).text
    soup = BeautifulSoup(html_doc, 'html.parser')
    article_list = soup.find_all(href=re.compile("news.yahoo.co.jp/"))
    for idx, article in enumerate(article_list):
        if "記事全文を読む" in str(article):
            article_url = article.attrs['href']
            break
        else:
            article_url = None
            if idx == len(article_list) - 1:
                return [np.nan, 'NaN']

    html_doc = requests.get(article_url).text
    soup = BeautifulSoup(html_doc, 'html.parser')

    try:
        comment_count = int(str(soup.find_all('div', {'class': "riff-VisuallyHidden__root"})).split('<!-- -->')[1])
    except IndexError:
        comment_count = 'NaN'
  
    ps = soup.find_all("p", {"class": "sc-jtggT bAhgUU yjSlinkDirectlink highLightSearchTarget"})
  
    texts = []
    for p in ps:
        texts.append(p.get_text())
    texts = [sentence.strip() for sentence in texts]  # 空白文字(スペースやタブ、改行)の削除
    texts = list(filter(lambda line: line != '', texts))
    texts = ''.join(texts)
    texts = texts.translate(str.maketrans(
          {'\n': '', '\t': '', '\r': '', '\u3000': '', ' ': ''}))  # 改行やタブ、全角スペースを消す
    return [texts, comment_count]

output = get_document(URLexample[0])
print(output[0])
print(output[1])

・全国的に冬本番の寒さ・北海道や東北は大雪のおそれ・北陸は雷雨や霰(あられ)に注意今日12月1日(木)は冬型の気圧配置が続き、上空に強い寒気が流れ込みます。全国的に気温が低く、冬本番の寒さとなる見込みです。北日本の日本海側を中心に大雪となるおそれがあります。強い寒気が日本列島に覆うため、全国的に冬本番の寒さになります。北海道は最高気温が0℃未満の真冬日の所が多い予想です。関東から西の各地も昼間の気温は10℃前後までしか上がらず、空気が冷たく感じます。関東は日差しが乏しく、寒い一日です。日本海で発達した雪雲が次々に流れ込み、北海道や東北北部の日本海側は断続的に雪が降ります。内陸部や山沿いを中心に積雪が増加する見込みです。車で移動をする場合は冬用タイヤを装着していても慎重な運転を行ってください。東北南部や北陸なども山沿いは積雪のおそれがあるため、注意が必要です。北陸は日本海でぶつかり合う風によって発達した雲の影響で雨が降りやすく、局地的に雷を伴って強く降ることがあります。また、活発な雨雲の周辺では霰(あられ)や突風にも注意が必要です。
141


In [31]:
from tqdm.notebook import tqdm

In [32]:
for genre in tqdm(genres):
    titles = list(concat_genre[genre].keys())
    urls = [i[0] for i in list(concat_genre[genre].values())]
    dates = [i[1] for i in list(concat_genre[genre].values())]
    texts = []
    comment_counts = []
    for url in tqdm(urls):
        texts.append(get_document(url)[0])
        comment_counts.append(get_document(url)[1])
  
    if genre == 'domestic':
        df = pd.DataFrame(columns=['genre', 'title', 'date', 'document', 'comment_count'])
        df['title'] = titles
        df['document'] = texts
        df['comment_count'] = comment_counts
        df['date'] = dates
        df['genre'] = genre
    else:
        df_genre = pd.DataFrame(columns=['genre', 'title', 'date', 'document'])
        df_genre['title'] = titles
        df_genre['document'] = texts
        df_genre['comment_count'] = comment_counts
        df_genre['date'] = dates
        df_genre['genre'] = genre
        df = pd.concat([df, df_genre])

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/69 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/121 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/87 [00:00<?, ?it/s]

In [50]:
display(df.head())

,genre,title,date,document,comment_count
0,domestic,流行語大賞「村神様」に決定,12/1(木) 14:10,1日、今年の「ユーキャン新語・流行語大賞」が発表され、年間大賞に「村神様」が選ばれた。【映像...,2166
1,domestic,全国的に気温低く 北日本は大雪も,12/1(木) 12:13,日本付近はあす2日（金）にかけて冬型の気圧配置が続き、北日本を中心に強い寒気が流れ込む見通し...,72
2,domestic,同性婚巡り 本質に踏み込む判決,12/1(木) 8:19,同性カップルが「家族になる制度」がないのは違憲状態――。同性カップルの婚姻をめぐり、東京地裁...,968
3,domestic,長射程ミサイル 同時開発を検討,12/1(木) 8:16,防衛省が「反撃能力」の保有に向け、検討している長射程ミサイル開発計画の概要がわかった。１０種...,NaN
4,domestic,敵基地攻撃能力 自公が実質合意,12/1(木) 7:51,政府の国家安全保障戦略など安保関連3文書の改定に向け、自民、公明両党の実務者協議が30日あり...,NaN


In [103]:
 FONT_PATH = "/usr/share/fonts/truetype/ipagp.ttf"
TXT_NAME = "/content/text"
 
def get_word_str(text):

    code_regex = re.compile('[\t\s!"#$%&\'\\\\()*+,-./:;；：<=>?@[\\]^_`{|}~○｢｣「」〔〕“”〈〉'\
    '『』【】＆＊（）＄＃＠？！｀＋￥¥％♪…◇→←↓↑｡･ω･｡ﾟ´∀｀ΣДｘ⑥◎©︎♡★☆▽※ゞノ〆εσ＞＜┌┘]')
    text = code_regex.sub('', text)
    #text = re.sub(r'[!-/:-@[-`{-~]', r' ', text)
    mecab = MeCab.Tagger('mecab-ipadic-neologd')
    parsed = mecab.parse(text)
    lines = parsed.split('\n')
    lines = lines[0:-2]
    word_list = []
 
    for line in lines:
        tmp = re.split('\t|,', line)
 
        # 名詞のみ対象
        if  "名詞" in tmp[4]:
            word_list.append(tmp[0])
 
    return word_list


def get_count(text: str) -> list:
    # 文字列取得
    word_list = get_word_str(text)
    #stopwordsの設定
    words = ["私","わたし","僕","あなた","みんな","ただ","ほか","それ", "もの", "の", "さん", "横井", "これ", "よう", "ところ","ため","うち","ここ","そう","どこ", "つもり", "いつ","あと","もん","はず","こと","そこ","あれ","なに","傍点","まま","事","人","方","何","時","一","二","三","四","五","六","七","八","九","十", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "１", "２", "３", "４", "５", "６", "７", "８", "９", "１０"]
    
    for word in words:
        if word in word_list:
            word_list = list(filter((word).__ne__, word_list))

    c = collections.Counter(word_list)
    list_item1 = []
    for i in list(c.items()):
        try:
            int(i[0])
        except ValueError:
            list_item1.append(i)
  
    list_item2 = []
    for i in list_item1:
        if len(i[0]) != 1:
            list_item2.append(i)
    return list_item2

In [111]:
df = df.reset_index(drop=True)
df_results = pd.DataFrame(columns=['genre', 'title', 'date', 'document', 'comment_count', 'word', 'count'])
for  idx, i in enumerate(range(df.shape[0])):
    df_result = pd.DataFrame(columns=['genre', 'title', 'date', 'document', 'comment_count', 'word', 'count'])
    df_ = df.iloc[[i]]
    genre = df_['genre'].values[0]
    title = df_['title'].values[0]
    date = df_['date'].values[0]
    text = df_['document'].values[0]
    comment_count = df_['comment_count'].values[0]
    if  text is not np.nan:
        word_list = get_count(text)
        words = [j[0]for j in word_list]
        count = [j[1]for j in word_list]
        df_result['word'] = words
        df_result['count'] = count
        df_result['genre'] = genre
        df_result['title'] = title
        df_result['date'] = date
        df_result['document'] = text
        df_result['comment_count'] = comment_count
        df_results = pd.concat([df_results, df_result], axis=0)

display(df_results.head(5))

,genre,title,date,document,comment_count,word,count
0,domestic,流行語大賞「村神様」に決定,12/1(木) 14:10,1日、今年の「ユーキャン新語・流行語大賞」が発表され、年間大賞に「村神様」が選ばれた。【映像...,2166,今年,1
1,domestic,流行語大賞「村神様」に決定,12/1(木) 14:10,1日、今年の「ユーキャン新語・流行語大賞」が発表され、年間大賞に「村神様」が選ばれた。【映像...,2166,ユー,1
2,domestic,流行語大賞「村神様」に決定,12/1(木) 14:10,1日、今年の「ユーキャン新語・流行語大賞」が発表され、年間大賞に「村神様」が選ばれた。【映像...,2166,キャン,1
3,domestic,流行語大賞「村神様」に決定,12/1(木) 14:10,1日、今年の「ユーキャン新語・流行語大賞」が発表され、年間大賞に「村神様」が選ばれた。【映像...,2166,新語,1
4,domestic,流行語大賞「村神様」に決定,12/1(木) 14:10,1日、今年の「ユーキャン新語・流行語大賞」が発表され、年間大賞に「村神様」が選ばれた。【映像...,2166,流行,1


In [112]:
df_results_copy = df_results.copy()
df_results_copy['genre'] = "all"
df_results = pd.concat([df_results, df_results_copy], axis=0)

In [135]:
df_results.sort_values('count', ascending=False)['word'][:30]

5      わかめ
5      わかめ
1     ラーメン
1     ラーメン
6       代表
6       代表
26      長田
26      長田
26      長田
26      長田
5     FIRE
5     FIRE
22     テレビ
22     テレビ
10      デモ
10      デモ
0       女性
20      バズ
20      バズ
11     タイヤ
0       女性
11     タイヤ
0       女性
20      バズ
0       女性
20      バズ
29      視聴
29      視聴
16      検査
16      検査
Name: word, dtype: object

In [ ]:
df_results.to_csv('result_yahoo.csv', index = None)